# Format & preprocess Neuroimaging datasets

## Transform your dataset into BIDS using Clinica


Before any processing, the OASIS dataset must be converted into a format friendly to neuroimaging tool. This format is called [BIDS](https://bids.neuroimaging.io/). A BIDS complient dataset consists in a specific organisation of the results obtained in neuroimaging experiment.

[Clinica provides tools](http://www.clinica.run/doc/Converters/OASIS2BIDS/) to generate automatically this organisation.

A command line instruction is enough to get the data in BIDS format:

```bash

clinica convert oasis-to-bids <dataset_directory> <clinical_data_directory> <bids_directory>
```

## Preprocess raw images with `t1-linear` pipeline

Original images are preprocessed following the pipeline described in the original [paper](https://www.sciencedirect.com/science/article/abs/pii/S1361841520300591?via%3Dihub). 
In this document, a "mininal" preprocessing procedure is described. This procedure performs the following operations:


* **Bias field correction** using the N4ITK method [Tustison et al., 2010].

* **Affine registration** to the MNI152NLin2009cSym template [Fonov et al., 2011, 2009] in MNI space with the SyN algorithm [Avants et al., 2008].

* **Cropping** the registered images in order to remove the background. The final image size is 169×208×179 with 1 mm3 isotropic voxels.


### Bias Field Correction.
MR images can be corrupted by a low frequency and smooth signal caused by magnetic field inhomogeneities. This bias field induces variations in the intensity of the same tissue in different locations of the image, which deteriorates the performance of image analysis algorithms such as registration (Vovk et al., 2007). Several methods exist to correct these intensity inhomogeneities, two popular ones being the nonparametric nonuniformity intensity normalization (N3) algorithm (Sled et al., 1998), available for example in the Freesurfer software package, and the N4 algorithm (Tustison et al., 2010) implemented in ITK.

### Affine registration
Medical image registration consists of spatially aligning two or more images, either globally (rigid and affine registration) or locally (non-rigid registration), so that voxels in corresponding positions contain comparable information. A large number of software tools have been developed for MRI-based registration, among them the SyN algorithm implemented in [ANTs](http://stnava.github.io/ANTs/). In this preprocessing, all images are aligned to a reference template (MNI152NLin2009cSym). 

### Cropping 
One images are aligned some unuseful voxels can be removed. For example, voxels representing the background do not contain preponderant information for the training process. On the contraty, removing the can improve computing efficiency. At the end these images have a size of 169×208×179 with 1 mm3 isotropic voxels.

These three steps are emmbeded in single pipeline provieded by Clinica. It is called [**T1_linear**](http://www.clinica.run/doc/Pipelines/T1_Linear/), and it is very simple to use:

```bash
clinica run t1-linear bids_directory caps_directory
```

The option `--crop_image` assures to perform the "cropping" stage. 

In [ ]:
!clinica run t1-linear bids_directory caps_directory

Once the pipeline has been run, the necessary outputs for the next steps are saved using a specific suffix: 
    `_space-MNI152NLin2009cSym_desc-Crop_res-1x1x1_T1w.nii.gz`. 

For example, processed images from our dataset are:

## Convert nifti files to tensors

Once the dataset had been preprocessed we need to obtain files suited for the training phase.
This task can be performed using the [Clinica pipeline `deeplearning-prepare-data`](http://www.clinica.run/doc/Pipelines/MachineLearning_Classification/)

This pipeline select the preprocessed images, extract "tensors", and write them as output files for the entire image, each slice ot each patch.

```
clinica run deeplearning-prepare-data <caps_directory> {image,slice,patch}
```

Output files are stored into a new folder (inside the CAPS) and follows a struture like this:

```
deeplearning_prepare_data
├── image_based
│   └── t1_linear
│       └── sub-<participant_label>_ses-<session_label>_T1w_space-MNI152NLin2009cSym_desc-Crop_res-1x1x1_T1w.pt
├── patch_basedd
│   └── t1_linear
│       ├── sub-<participant_label>_ses-<session_label>_T1w_space-MNI152NLin2009cSym_desc-Crop_res-1x1x1_patchsize-50_stride-50_patch-0_T1w.pt
│       ├── sub-<participant_label>_ses-<session_label>_T1w_space-MNI152NLin2009cSym_desc-Crop_res-1x1x1_patchsize-50_stride-50_patch-1_T1w.pt
│       ├── sub-<participant_label>_ses-<session_label>_T1w_space-MNI152NLin2009cSym_desc-Crop_res-1x1x1_patchsize-50_stride-50_patch-2_T1w.pt
│       ├── sub-<participant_label>_ses-<session_label>_T1w_space-MNI152NLin2009cSym_desc-Crop_res-1x1x1_patchsize-50_stride-50_patch-3_T1w.pt
│       ├── ...
│       ├── ...
│       ├── ...
│       └── sub-<participant_label>_ses-<session_label>_T1w_space-MNI152NLin2009cSym_desc-Crop_res-1x1x1_patchsize-50_stride-50_patch-N_T1w.pt
└── slice_based
    └── t1_linear
        ├── sub-<participant_label>_ses-<session_label>_T1w_space-MNI152NLin2009cSym_desc-Crop_res-1x1x1_axis-axi_channel-rgb_slice-0_T1w.pt
        ├── sub-<participant_label>_ses-<session_label>_T1w_space-MNI152NLin2009cSym_desc-Crop_res-1x1x1_axis-axi_channel-rgb_slice-1_T1w.pt
        ├── sub-<participant_label>_ses-<session_label>_T1w_space-MNI152NLin2009cSym_desc-Crop_res-1x1x1_axis-axi_channel-rgb_slice-2_T1w.pt
        ├── sub-<participant_label>_ses-<session_label>_T1w_space-MNI152NLin2009cSym_desc-Crop_res-1x1x1_axis-axi_channel-rgb_slice-3_T1w.pt
        ├── ... 
        ├── ...
        ├── ...
        ├── sub-<participant_label>_ses-<session_label>_T1w_space-MNI152NLin2009cSym_desc-Crop_res-1x1x1_axis-axi_channel-rgb_slice-N_T1w.pt
```

In a short, there is a folder for each feature (image, slice or patch) and inside the numerated tensor files with the corresponding feature. 